# Section 5: DataFrames in Depth

In [1]:
import numpy as np
import pandas as pd

## Introducting a New Dataset - Soccer

In this section we will be working with a Dataframe containing English Premier League soccer players. There are over 400 players and 17 attributes!

In [2]:
data_url = 'https://andybek.com/pandas-soccer'

Read in the data

In [3]:
players = pd.read_csv(data_url)

Let's take a look at the Dataframe info. Looks like we have quite a few numeric columns as well as object columns.

In [4]:
players.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Columns: 17 entries, name to new_signing
dtypes: float64(2), int64(10), object(5)
memory usage: 190.7 KB


We can take a higher level look at this using the `dtypes` attribute and calling the `value_counts()` method.

In [5]:
players.dtypes.value_counts()

int64      10
object      5
float64     2
dtype: int64

Take one last peek at the DataFrame structure to ensure we're ready to go.

In [6]:
players.head()

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2,France,0,4,1,1,0


## Quick Review - Indexing with boolean Masks

Recall the general approach for boolean indexing:
1. Generate a sequence of booleans ("Trues" and "Falses")
2. Use that boolean sequence with either square brackets [ ] or `.loc[]` to make the selection.

Say we're interested in learning which players have a market value exceeding 40 million dollars. We start by using the attribute accessor or selection brackets with a comparison operators to create the boolean sequence.

In [7]:
players["market_value"] > 40

0       True
1       True
2      False
3      False
4      False
       ...  
460    False
461    False
462    False
463    False
464    False
Name: market_value, Length: 465, dtype: bool

Now to select just the players with market value over 40 million, simply pass this expression into a set of selection brackets for the DataFrame itself

In [8]:
players[players["market_value"] > 40]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2,Germany,0,4,1,1,0
96,Eden Hazard,Chelsea,26,LW,1,75.0,4220,10.5,2.30%,224,2,Belgium,0,3,5,1,0
97,Diego Costa,Chelsea,28,CF,1,50.0,4454,10.0,3.00%,196,2,Spain,0,4,5,1,0
108,N%27Golo Kante,Chelsea,26,DM,2,50.0,4042,5.0,13.80%,83,2,France,0,3,5,1,1
218,Philippe Coutinho,Liverpool,25,AM,1,45.0,2958,9.0,30.80%,171,3,Brazil,0,3,10,1,0
244,Kevin De Bruyne,Manchester+City,26,AM,1,65.0,2252,10.0,17.50%,199,2,Belgium,0,3,11,1,0
245,Sergio Aguero,Manchester+City,29,CF,1,65.0,4046,11.5,9.70%,175,3,Argentina,0,4,11,1,0
246,Raheem Sterling,Manchester+City,22,LW,1,45.0,2074,8.0,3.80%,149,1,England,0,2,11,1,0
264,Romelu Lukaku,Manchester+United,24,CF,1,50.0,3727,11.5,45.00%,221,2,Belgium,0,2,12,1,0


Indeed, there are 13 players who meet this criteria.

In [9]:
players[players["market_value"] > 40].shape

(13, 17)

## More Approaches to Boolean Masking: `isin()`, `lt()`, `between()`

Say we're interested in looking at just the *Defenders* (soccer position) in the DataFrame. Defenders, or backs, can be left, center, or right. Let's see what the position codes are in our DataFrame by examining the `position` column and using the `unique()` method.


In [10]:
players['position'].unique()

array(['LW', 'AM', 'GK', 'RW', 'CB', 'RB', 'CF', 'LB', 'DM', 'RM', 'CM',
       nan, 'SS', 'LM'], dtype=object)

Out of those, we are interested in the defenders: LB, CB, and RB. Thus, we want to extract lines from our DataFrame on the condition that they are one of these three positions. This is a great use case for the *`isin()` method, which is used to create a boolean series that indicates whether that particular entry is in the list of values passed in.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html

In [11]:
players['position'].isin(['LB', 'CB', 'RB'])

0      False
1      False
2      False
3      False
4       True
       ...  
460    False
461     True
462     True
463    False
464    False
Name: position, Length: 465, dtype: bool

We can now apply the boolean mask to the DataFrame using `loc[]` to select just the Defenders. We see that 157 players in this list are defenders.

In [12]:
players.loc[players['position'].isin(['LB', 'CB', 'RB'])]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2,France,0,4,1,1,0
5,Hector Bellerin,Arsenal,22,RB,3,30.0,1675,6.0,13.70%,119,2,Spain,0,2,1,1,0
7,Nacho Monreal,Arsenal,31,LB,3,13.0,555,5.5,4.70%,115,2,Spain,0,4,1,1,0
8,Shkodran Mustafi,Arsenal,25,CB,3,30.0,1877,5.5,4.00%,90,2,Germany,0,3,1,1,1
17,Gabriel Paulista,Arsenal,26,CB,3,13.0,552,5.0,0.10%,45,3,Brazil,0,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,Aaron Cresswell,West+Ham,27,LB,3,12.0,380,5.0,1.30%,60,1,England,0,3,20,0,0
458,Angelo Ogbonna,West+Ham,29,CB,3,9.0,247,4.5,1.10%,45,2,Italy,0,4,20,0,0
459,Pablo Zabaleta,West+Ham,32,RB,3,7.0,698,5.0,2.70%,45,3,Argentina,0,5,20,0,0
461,Arthur Masuaku,West+Ham,23,LB,3,7.0,199,4.5,0.20%,34,4,Congo DR,0,2,20,0,1


Let's look at ranges of values. We can use the familiar `between()` method for this.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html
* Remember that by default, the edge values are inclusive. This can be changed using the `inclusive` parameter

Let's assess players that are between market values of 40 and 50 million dollars.

In [13]:
players.market_value.between(40, 50, inclusive = False)

0      False
1      False
2      False
3      False
4      False
       ...  
460    False
461    False
462    False
463    False
464    False
Name: market_value, Length: 465, dtype: bool

As before, we can now apply this boolean mask to the original DataFrame. it appears there are only three players swith a market value of above 40 million but below 50 million

In [14]:
players[players.market_value.between(40, 50, inclusive = False)]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
218,Philippe Coutinho,Liverpool,25,AM,1,45.0,2958,9.0,30.80%,171,3,Brazil,0,3,10,1,0
246,Raheem Sterling,Manchester+City,22,LW,1,45.0,2074,8.0,3.80%,149,1,England,0,2,11,1,0
380,Dele Alli,Tottenham,21,CM,2,45.0,4626,9.5,38.60%,225,1,England,0,1,17,1,0


If the edge values are inclusive, we see that the number of players is substantially higher.

In [15]:
players[players.market_value.between(40, 50, inclusive = True)]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2,Germany,0,4,1,1,0
31,Alexandre Lacazette,Arsenal,26,CF,1,40.0,1183,10.5,26.50%,0,2,France,1,3,1,1,0
97,Diego Costa,Chelsea,28,CF,1,50.0,4454,10.0,3.00%,196,2,Spain,0,4,5,1,0
102,Thibaut Courtois,Chelsea,25,GK,4,40.0,1260,5.5,18.50%,141,2,Belgium,0,3,5,1,0
108,N%27Golo Kante,Chelsea,26,DM,2,50.0,4042,5.0,13.80%,83,2,France,0,3,5,1,1
218,Philippe Coutinho,Liverpool,25,AM,1,45.0,2958,9.0,30.80%,171,3,Brazil,0,3,10,1,0
219,Sadio Mane,Liverpool,25,LW,1,40.0,3219,9.5,5.30%,156,4,Senegal,0,3,10,1,1
246,Raheem Sterling,Manchester+City,22,LW,1,45.0,2074,8.0,3.80%,149,1,England,0,2,11,1,0
263,Bernardo Silva,Manchester+City,22,RW,1,40.0,1098,8.0,4.60%,0,2,Portugal,1,2,11,1,0
264,Romelu Lukaku,Manchester+United,24,CF,1,50.0,3727,11.5,45.00%,221,2,Belgium,0,2,12,1,0


How else can we create boolean Series on the fly? One way is to use old-fashioned comparison operator. For instance, if we want to look at players age 25 or younger: 

In [16]:
players['age'] <= 25

0      False
1      False
2      False
3      False
4      False
       ...  
460     True
461     True
462     True
463     True
464    False
Name: age, Length: 465, dtype: bool

In [17]:
players[players['age'] <= 25]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
5,Hector Bellerin,Arsenal,22,RB,3,30.0,1675,6.0,13.70%,119,2,Spain,0,2,1,1,0
8,Shkodran Mustafi,Arsenal,25,CB,3,30.0,1877,5.5,4.00%,90,2,Germany,0,3,1,1,1
9,Alex Iwobi,Arsenal,21,LW,1,10.0,1812,5.5,1.00%,89,4,Nigeria,0,1,1,1,0
10,Granit Xhaka,Arsenal,24,DM,2,35.0,1815,5.5,2.00%,85,2,Switzerland,0,2,1,1,0
11,Granit Xhaka,Arsenal,24,DM,2,35.0,1815,5.5,2.00%,85,2,Switzerland,0,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Pedro Obiang,West+Ham,25,CM,2,9.0,286,4.5,0.30%,55,2,Spain,0,3,20,0,0
460,Edimilson Fernandes,West+Ham,21,CM,2,5.0,288,4.5,0.40%,38,2,Switzerland,0,1,20,0,1
461,Arthur Masuaku,West+Ham,23,LB,3,7.0,199,4.5,0.20%,34,4,Congo DR,0,2,20,0,1
462,Sam Byram,West+Ham,23,RB,3,4.5,198,4.5,0.30%,29,1,England,0,2,20,0,0


However, Pandas also has another method for accomplishing this directly. Instead of the `<=` method, we can use the `.le()` method, which stands for "less than or equal to". 

In [18]:
players[players['age'].le(25)]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
5,Hector Bellerin,Arsenal,22,RB,3,30.0,1675,6.0,13.70%,119,2,Spain,0,2,1,1,0
8,Shkodran Mustafi,Arsenal,25,CB,3,30.0,1877,5.5,4.00%,90,2,Germany,0,3,1,1,1
9,Alex Iwobi,Arsenal,21,LW,1,10.0,1812,5.5,1.00%,89,4,Nigeria,0,1,1,1,0
10,Granit Xhaka,Arsenal,24,DM,2,35.0,1815,5.5,2.00%,85,2,Switzerland,0,2,1,1,0
11,Granit Xhaka,Arsenal,24,DM,2,35.0,1815,5.5,2.00%,85,2,Switzerland,0,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Pedro Obiang,West+Ham,25,CM,2,9.0,286,4.5,0.30%,55,2,Spain,0,3,20,0,0
460,Edimilson Fernandes,West+Ham,21,CM,2,5.0,288,4.5,0.40%,38,2,Switzerland,0,1,20,0,1
461,Arthur Masuaku,West+Ham,23,LB,3,7.0,199,4.5,0.20%,34,4,Congo DR,0,2,20,0,1
462,Sam Byram,West+Ham,23,RB,3,4.5,198,4.5,0.30%,29,1,England,0,2,20,0,0


The result is equivalent to using comparison operators. To prove this, let's use the `equals()` method to compare them.

In [19]:
players.age.le(25).equals(players.age <= 25)

True

There are multiple comparison methods are your disposal:
* `le()`: less than or equal to (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.le.html)
* `gt()`: greater than (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.gt.html)
* `lt()`: less than (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.lt.html)
* `ge()`: greater than or equal to (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.ge.html)
* `ne()`: not equal to (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.ne.html)
* `eq()`: equal to (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.eq.html)

The main difference between using comparators and using these comparison methods is that the methods have a `fill_value` parameter that can be used to substitute in missing data.

## Binary Operators with Booleans

Before we get to combining conditions, let's look at how booleans behave in isolation

**Binary operators** are similar to other operators, but they work on the binary representation of the values, that is, the individual bits. They allow us to operations and comparisons or complements on a bit-by-bit basis. They are also known as bit-wise operators.

The most useful binary operators are `OR` and `AND`

Let's first examine `OR`. What would we expect if we performed a binary operator of False OR True (with OR represented by the pipe `|`.

We expect to get True, because an OR comparison will resolve to True as long as at least one of the conditions is True. Think of the `OR` operator as "searching for True"

In [20]:
True | False

True

Let's try some others

In [21]:
False | False

False

In [22]:
True | True

True

Let's now look at the `AND` operator, represented by the ampersand `&`. Think of the AND operator as always True unless there is a False.
* A single `False` is enough to trigger a "False".

In [23]:
False & True

False

In [24]:
True & True

True

In [25]:
False & False

False

In [26]:
True & False

False

In [27]:
True & True & False & True & True

False

How do Pandas Series combine using booleans? Let's start by making a single-element series containing a single False value

In [28]:
f = pd.Series(False)

Let's also make a single-value Series that contains True

In [29]:
t = pd.Series(True)

Let's now use the AND and OR operators on them. What we get is a combined series that compares the boolean values of each Series

In [30]:
t & f

0    False
dtype: bool

In [31]:
t | f

0    True
dtype: bool

That was simple, but where this becomes very powerful is with long series of booleans.

In [32]:
t = pd.Series([True if i % 2 == 0 else False for i in range(10)])

In [33]:
t

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
9    False
dtype: bool

In [34]:
f = pd.Series([False for i in range(10)])

In [35]:
f

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

What happens when we combine these together using the AND and OR operators? 

The `&` operator, as expected, returns a series of all False values since the `f` series consists of all False values.

In [36]:
t & f

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

With the `|` OR operator, the result will be alternating `True` and `False`, as we would expect since the `t` Series contains alternating True and False values

In [37]:
t | f

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
9    False
dtype: bool

The important thing to note is that when we combine two Pandas series, the comparison is done **label-to-label**. It is NOT done based on the order. Consider this example:

In [38]:
f = pd.Series(data = [False, True, True], index = ['c', 'b', 'a'])
t = pd.Series(data = [True, False, False], index = ['a','b','c'])

In [39]:
f

c    False
b     True
a     True
dtype: bool

In [40]:
t

a     True
b    False
c    False
dtype: bool

Now let's use the `&` operator. If the comparison were based on order of the values within the individual Series, we would expect `[False, False, False]`. 

In [41]:
f & t

a     True
b    False
c    False
dtype: bool

We did not get that! Instead, we got `True` for `a` and `False` for `b` and `c`. That's because the comparison was done by the index labels `a`, `b`, and `c` and NOT the positions of the values.

This will help illustrate what happens behind the scenes when we combine booleans. Onward and upward!

## BONUS - XOR and Complement Binary Ops

The binary OR and AND operators are without a doubt the most frequently used ones. However, there are several others available. This lecture will cover two of them.

XOR stands for "exclusive or". As the name suggests, it is exclusive. That means it resolves to *True* when the inputs are different, and to *False* if they are alike.
* In Python the XOR operator is represented by `^`

In [42]:
True ^ False

True

In [43]:
False ^ False

False

In [44]:
True ^ True

False

In [45]:
True ^ (False | False & True) | False

True

XOR can be used to combine boolean series in DataFrame indexing, such as when we want one condition but not the other. We'll see examples of this later.


Another binary operator is the **complement operator**, represented by the tilde `~`.
* In Pandas and Numpy it is extremely useful for negating boolean series.

In [46]:
~False

-1

In [47]:
~True

-2

What's going on here? It has to do with how computers represents integers. The "twos complement" system is a scheme for deriving binary representations of integer numbers. It is the most common method for representing signed integers for computers.
Remember that computers only understand 0's and 1's. How then do we work with integers? We use binary numbers and their respective two's complement. The two's complement is the inverstion of the binary representation of the number.
* https://en.wikipedia.org/wiki/Two%27s_complement
* https://www.cs.cornell.edu/~tomf/notes/cps104/twoscomp.html
* Simple tutorial on binary numbers: http://www.steves-internet-guide.com/binary-numbers-explained/

Going back to our previous example, we know that the integer value of False is 0 while the integer value of True is 1. Therefore:
* `~True = ~1 = inversion(00000001) = 11111110 = -2`
* `~False = ~0 = inversion(00000000) = 11111111 = -1`

Going back to Pandas, let's use the complement binary operator to negate our binary operators. It will turn our Trues to Falses and vice versa.

In [48]:
t = pd.Series([True, True, False])

In [49]:
t

0     True
1     True
2    False
dtype: bool

In [50]:
~t

0    False
1    False
2     True
dtype: bool

The result of using the `~` here is a negation of each of our boolean values. This feature will be very useful for defining negative conditions with DataFrames. For example, if we wanted to select all soccer players that are NOT defenders.

In [51]:
players

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2,France,0,4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,Edimilson Fernandes,West+Ham,21,CM,2,5.0,288,4.5,0.40%,38,2,Switzerland,0,1,20,0,1
461,Arthur Masuaku,West+Ham,23,LB,3,7.0,199,4.5,0.20%,34,4,Congo DR,0,2,20,0,1
462,Sam Byram,West+Ham,23,RB,3,4.5,198,4.5,0.30%,29,1,England,0,2,20,0,0
463,Ashley Fletcher,West+Ham,21,CF,1,1.0,412,4.5,5.90%,16,1,England,0,1,20,0,1


## Combining Conditions - Indexing with Multiple Conditions

Quick refresher, let's select all of the LBs (left backs)

In [52]:
players[players.position == 'LB']

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
7,Nacho Monreal,Arsenal,31,LB,3,13.00,555,5.5,4.70%,115,2,Spain,0,4,1,1,0
18,Kieran Gibbs,Arsenal,27,LB,3,10.00,489,5.0,0.50%,45,1,England,0,3,1,1,0
29,Sead Kolasinac,Arsenal,24,LB,3,15.00,618,6.0,6.90%,0,2,Bosnia,1,2,1,1,0
34,Charlie Daniels,Bournemouth,30,LB,3,3.00,185,5.0,19.80%,134,1,England,0,4,2,0,0
54,Brad Smith,Bournemouth,23,LB,3,2.00,297,4.0,3.30%,4,4,Australia,0,2,2,0,0
62,Gaetan Bong,Brighton+and+Hove,29,LB,3,1.50,97,4.5,0.20%,0,4,Cameroon,0,4,3,0,0
65,Markus Suttner,Brighton+and+Hove,30,LB,3,2.00,23,4.5,0.20%,0,2,Austria,0,4,3,0,0
82,Stephen Ward,Burnley,31,LB,3,1.50,152,4.5,2.50%,91,2,Ireland,0,4,4,0,0
99,Marcos Alonso Mendoza,Chelsea,26,LB,3,25.00,3069,7.0,12.40%,177,2,Spain,0,3,5,1,1
112,Kenedy,Chelsea,21,LB,3,7.00,566,5.0,0.10%,3,3,Brazil,0,1,5,1,0


Now, what if we want to restrict by yet another condition? Simple! Just use the binary operators that we have learned.
For instance, say we want all of the left backs who are 25 years old or younger. We can place both conditionals within the square selection brackets and combine them using the `&` operator.
* **IMPORTANT NOTE**: Always wrap your individual conditions within square brackets. Failure to do so will cause Python to misinterpret your conditions due to order of operations.

In [53]:
players[(players.position == 'LB') & (players.age <= 25)]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
29,Sead Kolasinac,Arsenal,24,LB,3,15.00,618,6.0,6.90%,0,2,Bosnia,1,2,1,1,0
54,Brad Smith,Bournemouth,23,LB,3,2.00,297,4.0,3.30%,4,4,Australia,0,2,2,0,0
112,Kenedy,Chelsea,21,LB,3,7.00,566,5.0,0.10%,3,3,Brazil,0,1,5,1,0
128,Jeffrey Schlupp,Crystal+Palace,24,LB,3,8.00,385,5.0,0.30%,47,4,Ghana,0,2,6,0,0
212,Ben Chilwell,Leicester+City,20,LB,3,2.50,288,4.5,0.80%,19,1,England,0,1,9,0,0
236,Alberto Moreno,Liverpool,25,LB,3,10.00,397,4.5,0.30%,8,2,Spain,0,3,10,1,0
281,Luke Shaw,Manchester+United,22,LB,3,20.00,947,5.0,0.40%,45,1,England,0,2,12,1,0
294,Paul Dummett,Newcastle+United,25,LB,3,3.50,177,4.5,1.00%,0,2,Wales,0,3,13,0,0
298,Massadio Haidara,Newcastle+United,24,LB,3,1.50,114,4.0,0.50%,0,2,France,0,2,13,0,0
328,Matt Targett,Southampton,21,LB,3,3.00,110,4.5,0.20%,12,1,England,0,1,14,0,0


You could even add more conditions! Let's continue filtering for LBs age 25 or younger that have a market value of at least $10 million

In [54]:
players[(players.position == 'LB') & (players.age <= 25) & (players.market_value >= 10)]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
29,Sead Kolasinac,Arsenal,24,LB,3,15.0,618,6.0,6.90%,0,2,Bosnia,1,2,1,1,0
236,Alberto Moreno,Liverpool,25,LB,3,10.0,397,4.5,0.30%,8,2,Spain,0,3,10,1,0
281,Luke Shaw,Manchester+United,22,LB,3,20.0,947,5.0,0.40%,45,1,England,0,2,12,1,0
389,Ben Davies,Tottenham,24,LB,3,12.0,396,5.5,1.80%,90,2,Wales,0,2,17,1,0


Very cool. We can also combine different binary operators. For example, let's exclude this subset of left backs who are not from Arsenal or Tottenham. One way to do this is to use the `isin()` method and pass in "Arsenal" and "Tottenham", and then negate it using the tilde `~`

In [55]:
players[(players.position == 'LB') & (players.age <= 25) & (players.market_value >= 10) & (~players.club.isin(['Tottenham', 'Arsenal']))]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
236,Alberto Moreno,Liverpool,25,LB,3,10.0,397,4.5,0.30%,8,2,Spain,0,3,10,1,0
281,Luke Shaw,Manchester+United,22,LB,3,20.0,947,5.0,0.40%,45,1,England,0,2,12,1,0


## Conditions As Variables

The long indexing that combines multiple conditions can get very ugly to look at very quickly. One way to make the code more readable (aside from breaking into separate lines) is to use *conditions as variables*. 

This essentially means refactoring your conditions into standalone variables.

Suppose we are interested in right backs (RBs) from Arsenal and goalkeepers from Chelsea. Start by creating a variable generating a boolean mask for Arsenal players

In [56]:
arsenal_player = players.club == 'Arsenal'

In [57]:
arsenal_player

0       True
1       True
2       True
3       True
4       True
       ...  
460    False
461    False
462    False
463    False
464    False
Name: club, Length: 465, dtype: bool

Next, let's get the right backs from Arsenal

In [58]:
right_back = players.position == 'RB'

In [59]:
right_back

0      False
1      False
2      False
3      False
4      False
       ...  
460    False
461    False
462     True
463    False
464    False
Name: position, Length: 465, dtype: bool

For the Chelsea part, we'll combine both conditions in one go.

In [60]:
chelsea_goalkeepers = (players.club == 'Chelsea') & (players.position == 'GK')

In [61]:
chelsea_goalkeepers

0      False
1      False
2      False
3      False
4      False
       ...  
460    False
461    False
462    False
463    False
464    False
Length: 465, dtype: bool

Finally, let's index the whole DataFrame based on the conditions we have defined in these variables. To switch things up a bit we'll use `.loc[]`

In [65]:
players.loc[(arsenal_player & right_back) | chelsea_goalkeepers]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
5,Hector Bellerin,Arsenal,22,RB,3,30.0,1675,6.0,13.70%,119,2,Spain,0,2,1,1,0
27,Carl Jenkinson,Arsenal,25,RB,3,5.0,561,4.5,0.40%,2,1,England,0,3,1,1,0
102,Thibaut Courtois,Chelsea,25,GK,4,40.0,1260,5.5,18.50%,141,2,Belgium,0,3,5,1,0
109,Willy Caballero,Chelsea,35,GK,4,1.5,542,5.0,0.20%,64,3,Argentina,0,6,5,1,0


## Skill Challenge
Identify the subset of players that meets all of the following criteria:
1. English `nationality`
2. Market value is more than twice the average market value in the league (`market_value`)
3. More than 4000 page views (`page_views`) OR are a new signing (`new_signing`), but not both.

Let's handle this by creating variables for conditions. Let's start with English-ness

In [66]:
english = players.nationality == "England"

Next let's handle market value. First we'll calculate the average market value for all players, and then create a condition where only players that are more than twice the average market value are selected.

In [67]:
avg_market = players['market_value'].mean()

In [68]:
avg_market

11.125649350649349

In [71]:
twice_avg = players.market_value > avg_market * 2

In [72]:
twice_avg

0       True
1       True
2      False
3      False
4      False
       ...  
460    False
461    False
462    False
463    False
464    False
Name: market_value, Length: 465, dtype: bool

Finaly, let's handle the page views and signing condition. Since we want players that either have >4000 page views OR are a new_signing, but not both, our selections must reflect that.
That is, our 4000 views selector must exclude new_signing, and our signing selector must exclude players greater than 4000 views

In [87]:
fourK_views = players['page_views'] > 4000

In [88]:
new_signing = players['new_signing'] == 1

We now combine all of the expressions together. The third condition (views and signing) is a great opportunity to implement the EXOR operator `^` when combinding them. Remember that this will only resolve to True if either one of the conditions is True but not both. That's exactly what we want.

In [91]:
players[english & twice_avg & (fourK_views ^ new_signing)]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
256,John Stones,Manchester+City,23,CB,3,35.0,1078,5.5,2.30%,59,1,England,0,2,11,1,1
380,Dele Alli,Tottenham,21,CM,2,45.0,4626,9.5,38.60%,225,1,England,0,1,17,1,0
381,Harry Kane,Tottenham,23,CF,1,60.0,4161,12.5,35.10%,224,1,England,0,2,17,1,0


## Two-Dimensional Indexing: Selecting Columns


So far, all of our data extractions have operated only on the index axis - that is, selecting *rows* that we want.
However, remember that DataFrames are two-dimensional data structures. Thus, we can index by columns as well.

Say we want to select Chelsea players that are age 23 and under. We already know how to do this.

In [94]:
chelsea_23under = (players.club == "Chelsea") & (players.age.le(23) )

In [96]:
players.loc[chelsea_23under]

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
110,Michy Batshuayi,Chelsea,23,CF,1,25.0,1162,8.5,1.60%,48,2,Belgium,0,2,5,1,1
111,Kurt Zouma,Chelsea,22,CB,3,15.0,723,5.5,0.80%,15,2,France,0,2,5,1,0
112,Kenedy,Chelsea,21,LB,3,7.0,566,5.0,0.10%,3,3,Brazil,0,1,5,1,0
115,Tiemoue Bakayoko,Chelsea,22,DM,2,16.0,1011,5.0,1.60%,0,2,France,1,2,5,1,0


But notice that these types of selections return ALL columns. What if we were only interested in one or a few columns, but not all of them?

Using the `loc[]` indexer, we can pass in a second argument that represents our columns that we want to select. 
* https://datagy.io/pandas-select-columns/#loc-select-columns

This is perhaps the most intuitive way to do it, but it is not the most flexible. For example, it won't allow you to select all columns that start with a particular letter.


In [97]:
players.loc[chelsea_23under, ['position', 'market_value']]

,position,market_value
110,CF,25.0
111,CB,15.0
112,LB,7.0
115,DM,16.0


Let's actually try that. Let's select all columns that begin with a 'p'. To do this, we can use the `startswith` string method on the columns attribute for the DataFrame, which generates a boolean mask for the columns! 

**Important note**: just like selecting rows, selecting columns with a boolean mask requires the boolean series to be of the same length as the column axis. 

In [99]:
startswith_p = players.columns.str.startswith('p')

Now we can use that boolean mask as the second argument for `loc[]`. Viola, we get just the columns that start with `p`.

In [100]:
players.loc[chelsea_23under, startswith_p]

,position,position_cat,page_views
110,CF,1,1162
111,CB,3,723
112,LB,3,566
115,DM,2,1011


It is also common to chain two square brackets together. The instructor suggests avoiding doing this.

In [104]:
players[chelsea_23under]['position']

110    CF
111    CB
112    LB
115    DM
Name: position, dtype: object

What you are essentially doing above is subsetting your DataFrame by the first condition, and then selecting a single column from that.

This is NOT the same as the following:

In [106]:
players.loc[chelsea_23under, 'position']

110    CF
111    CB
112    LB
115    DM
Name: position, dtype: object

Although the output is the same, behind the scenes the square bracket method-chaining is a slower process. This is because a `__getitem__` method under the hood which gets called twice with  bracket chaining.
Therefore, the instructor recomments avoiding bracket chaining and instead using `loc[]` with a second argument.